In [1]:
import parse
import analyse
import matplotlib.pyplot as plt
import numpy as np

In [2]:
benchmarks = [
"results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05",
"results/ACASXU_run2a_1_1_batch_2000-student-1-0.05",
"results/ACASXU_run2a_1_2_batch_2000-retrain-1-0.05",
"results/ACASXU_run2a_1_2_batch_2000-student-1-0.05",
"results/mnist8x8_100_80_60_40_20_10_eps1-9000.2-15",
"results/mnist8x8_student_36_10-9000.7-top",
"results/mnist8x8_student_12_12_12_10-9000.7-top",
"results/mnist8x8_100_80_60_40_20_10-9200.3-top",
"results/mnist8x8_100_200_300_200_100_10-9200.2-top"
]
variants = [
    'REFINE_UNTIL_LAST_OPTIMISTIC1',
    'CEGAR_OPTIMAL'
]
commit = '3ad4254'
out_handlers=[parse.CegarOptimal]
err_handlers=[parse.RunLim]
both_handlers=[parse.NNEquivResultHandler]
num_runs=1

In [3]:
variant_runs = {}
for v in variants:
    variant_runs[v]={}
    for b in benchmarks:
        variant_runs[v][b]=[]
        for i in range(1,num_runs+1):
            try:
                variant_runs[v][b].append(
                    parse.BenchmarkRun(
                        "../"+b+"-"+v+"/"+str(i)+"/nnequiv-"+commit+"/stdout.log",
                        "../"+b+"-"+v+"/"+str(i)+"/nnequiv-"+commit+"/stderr.log",
                        out_handlers=out_handlers,
                        err_handlers=err_handlers,
                        both_handlers=both_handlers
                    )
                )
            except FileNotFoundError:
                print(f"Count not find "+b+"-"+v+"/"+str(i)+"/nnequiv-"+commit+"/stdout.log")

In [11]:
variants = [
    '',
]
commit = '04f3bfb'
out_handlers=[]
err_handlers=[parse.RunLim]
both_handlers=[parse.BueningResultHandler]
num_runs=3

In [12]:
for v in variants:
    variant_runs[v]={}
    for b in benchmarks:
        variant_runs[v][b]=[]
        for i in range(1,num_runs+1):
            try:
                variant_runs[v][b].append(
                    parse.BenchmarkRun(
                        "../"+b+"-"+v+"/"+str(i)+"/buening-"+commit+"/stdout.log",
                        "../"+b+"-"+v+"/"+str(i)+"/buening-"+commit+"/stderr.log",
                        out_handlers=out_handlers,
                        err_handlers=err_handlers,
                        both_handlers=both_handlers
                    )
                )
            except (FileNotFoundError,Exception):
                print(f"Count not find "+b+"-"+v+"/"+str(i)+"/buening-"+commit+"/stdout.log")

Did not get all runlim values!
Count not find results/ACASXU_run2a_1_2_batch_2000-retrain-1-0.05-/3/buening-04f3bfb/stdout.log


In [13]:
variants = [
    'REFINE_UNTIL_LAST_OPTIMISTIC1',
    'CEGAR_OPTIMAL',
    ''
]
variants_median = {}
for v in variants:
    variants_median[v]={}
    for b in benchmarks:
        variants_median[v][b]=parse.AggregateRun(variant_runs[v][b])

print(variants_median["REFINE_UNTIL_LAST_OPTIMISTIC1"]["results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05"].get_val(['runlim','real']))
print(variants_median[""]["results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05"].get_val(['runlim','real']))
print(variants_median["noop"]["results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05"].get_val(['runlim','real']))

167.45
10800.1


KeyError: 'noop'

In [14]:
variants_median.keys()

dict_keys(['REFINE_UNTIL_LAST_OPTIMISTIC1', 'CEGAR_OPTIMAL', ''])

In [15]:
import pandas as pd

In [16]:
benchmarks_labels = {
"results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05":"ACAS_1_1-retrain",
"results/ACASXU_run2a_1_1_batch_2000-student-1-0.05":"ACAS_1_1-student",
"results/ACASXU_run2a_1_2_batch_2000-retrain-1-0.05":"ACAS_1_2-retrain",
"results/ACASXU_run2a_1_2_batch_2000-student-1-0.05":"ACAS_1_2-student",
"results/mnist8x8_100_80_60_40_20_10_eps1-9000.2-15":"MNIST_large-epsilon",
"results/mnist8x8_student_36_10-9000.7-top":"MNIST_small-top",
"results/mnist8x8_student_12_12_12_10-9000.7-top":"MNIST_medium-top",
"results/mnist8x8_100_80_60_40_20_10-9200.3-top":"MNIST_large-top",
"results/mnist8x8_100_200_300_200_100_10-9200.2-top":"MNIST_larger-top"
}

#columns = ["Name", "Status", "Equiv", "NNEquiv", "Kleine Büning"]
columns = ["Name", "NNEquiv", "NNEquivOpt", "Kleine Büning"]
data = []
for b in benchmarks:
    cur_row = []
    cur_row.append(benchmarks_labels[b])
    #cur_row.append("")
    #cur_row.append("")
    #cur_row[1]+="/"+str(variants_median[v][b].get_val(['result','status'],strategy='A')[0])
    #cur_row[2]+="/"+str(variants_median[v][b].get_val(['result','is_equiv'],strategy='A')[0])
    cur_row.append(variants_median['REFINE_UNTIL_LAST_OPTIMISTIC1'][b].get_val(['runlim','real']))
    cur_row.append(variants_median['CEGAR_OPTIMAL'][b].get_val(['cegaropt','opt']))
    cur_row.append(variants_median[''][b].get_val(['runlim','real']))
    data.append(cur_row)
print(columns)
table = pd.DataFrame(data, columns=columns)
table

['Name', 'NNEquiv', 'NNEquivOpt', 'Kleine Büning']


,Name,NNEquiv,NNEquivOpt,Kleine Büning
0,ACAS_1_1-retrain,167.45,51.483027,10800.100
1,ACAS_1_1-student,74.52,28.728671,10800.120
2,ACAS_1_2-retrain,315.13,102.306155,10800.115
3,ACAS_1_2-student,105.18,41.018371,320.070
4,MNIST_large-epsilon,33.43,14.774420,19.970
5,MNIST_small-top,14.39,6.223243,3.510
6,MNIST_medium-top,96.41,50.790681,3.850
7,MNIST_large-top,11.41,5.348326,25.850
8,MNIST_larger-top,715.87,334.426940,386.040


In [17]:
print(table.to_latex(index=False))

\begin{tabular}{lrrr}
\toprule
                Name &  NNEquiv &  NNEquivOpt &  Kleine Büning \\
\midrule
    ACAS\_1\_1-retrain &   167.45 &   51.483027 &      10800.100 \\
    ACAS\_1\_1-student &    74.52 &   28.728671 &      10800.120 \\
    ACAS\_1\_2-retrain &   315.13 &  102.306155 &      10800.115 \\
    ACAS\_1\_2-student &   105.18 &   41.018371 &        320.070 \\
 MNIST\_large-epsilon &    33.43 &   14.774420 &         19.970 \\
     MNIST\_small-top &    14.39 &    6.223243 &          3.510 \\
    MNIST\_medium-top &    96.41 &   50.790681 &          3.850 \\
     MNIST\_large-top &    11.41 &    5.348326 &         25.850 \\
    MNIST\_larger-top &   715.87 &  334.426940 &        386.040 \\
\bottomrule
\end{tabular}

